In [37]:
# User Input
#    ↓
# Enricher LLM → parses natural language query into smart intent & goal metadata
#    ↓
# Orchestrator LLM → decides what tools to call (MarketDataTool, NewsTool, etc.)
#    ↓
# Parallel Tool Execution (runs only the selected ones)
#    ↓
# Synthesizer LLM → builds final human-readable report
#    ↓
# ✅ Done


In [38]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [39]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="Gemma2-9b-It")

In [152]:
from typing_extensions import TypedDict
from pydantic import BaseModel , Field

class Refined(BaseModel):
    entities : list[str] = Field(description = "entities mentioned if any" )
    intent  : str = Field(description = "intent of the query")
    goal    : str = Field(description = "goal of the query")
    detail_level : str = Field(default="Basic" , description = "detail level what user demands.")
    general_search_query: str = Field(default=None, description="If the query is general, provide a well-phrased search query string to use directly for a general search.")

class ToolToCall(BaseModel):
    name : str = Field( description = "Name of the tool to call" )
    reason : str = Field( description = "contains reason why should this tool be called" )
    input : str

class ToolSchema(BaseModel):
    name: str
    description: str
    input_schema: str

class MessageState(TypedDict):

    user_query : str
    refined_query : Refined
    tools_to_call : list[ToolToCall]

In [153]:
from langchain_core.messages import HumanMessage , SystemMessage

def refining_query(state : MessageState):
    prompt = [
        SystemMessage( content = "You are a smart assistant that extracts user intent from stock market related queries." ),
        HumanMessage( content = f"""
        User asked : { state["user_query"] }
        
        Return structured content in form of :

          entities : entities if mentioned 
          intent  : intent of the query
          goal : goal of the query
          detail_level : detail level what user demands.
          general_search_query: If the user's query is general (like asking about overall market situation), create a polished, complete English search query that could be used directly in a web search.

        """ )
    ]
    structured_llm = llm.with_structured_output(Refined)
    response = structured_llm.invoke(prompt)
    return {"refined_query" : response}
    

In [154]:
input_query = {"user_query" : "Tell me about the Apple and Xiomi stock market today , also how is market now days "}
refined_query = refining_query(input_query)
print(refined_query)

{'refined_query': Refined(entities=['Apple', 'Xiomi'], intent='query', goal='stock information', detail_level='Basic', general_search_query=None)}


# TOOLS 

SocialMedia Tool

In [155]:
from langchain_community.tools.tavily_search import TavilySearchResults

def SocialMediaTool(entity):

    """
    Collect social media posts about a company or entity from platforms like Reddit, Twitter, and StockTwits.

    Args   : 
        entity : str
    Return :
        social media content : str
    """

    search_tool = TavilySearchResults(max_results=15)


    results = search_tool.invoke(f"""
                site:reddit.com OR site:twitter.com OR site:stocktwits.com {entity}" 
    """)

    content = []

    for r in results:
        if r.get('content') and len(r.get('content').strip()) > 100:
            content.append(
                {
                    "name" : r.get('title') ,
                    "reason" : r.get('content')
                }
            )
        
    return content


In [156]:

socialcontent = SocialMediaTool("Apple")
print(socialcontent)

[{'name': 'Apple iOS - Reddit', 'reason': "Apple is using some type of inline autocorrect that can't be disabled even if autocorrect is disabled in settings. The result is that words are changed after"}, {'name': 'r/Apple - Reddit', 'reason': 'All posts must be directly related to Apple. We will remove posts that are only vaguely related to Apple.\n\n\n8\nNo tech-support questions.\n\n\nAs per our Content Philosophy, this subreddit allows content that benefits the community over individual questions. As such, tech support threads and "What Should I Buy?" threads will be removed. Users will be sent to the "Daily Advice Thread" that\'s posted every day by Automoderator at 6AM EST. [...] About Reddit Advertise Help Blog Careers*   Press\n\n\n\n\nCommunities Best of Reddit Topics [...] Top 1% Rank by size\nCreated Jan 25, 2008\nPublic\nAnyone can view, post, and comment to this community\n\nCommunity Bookmarks\n\nApple\'s Website\nPromoted\n\n\nContent Philosophy'}, {'name': 'Is anyone usi

GeneralSearch Tool

In [157]:
from langchain_community.tools.tavily_search import TavilySearchResults

def GeneralSearchTool(query : str) -> str:

    """
    Perform a general web search to gather broader information on any topic.

    Args   : 
        entity : str
    Return :
        content : str 
    """

    search_tool = TavilySearchResults(max_results=15)

    results = search_tool.run(query)

    content = []

    for r in results:
        if r.get('content') and len(r.get('content').strip()) > 100:
            content.append(
                {
                    "name" : r.get('title') ,
                    "reason" : r.get('content')
                }
            )
        
    return content


In [158]:
GeneralSearchContent = GeneralSearchTool("Hi tell me about curret stock market")
print(GeneralSearchContent)

[{'name': 'Stock Market Today: Stock Market News And Analysis', 'reason': "Stock Market News And Analysis\nThe analysis you'll find in the Stock Market Today is based on over 130 years of market history and a detailed study of every top-performing stock since the 1880s.\nBy tracking the current recommended market exposure level, the Stock Market Today helps you stay on top of the latest stock market news while keeping any latest fluctuations in perspective to help you decide what action, if any, to take. [...] Stock Market Today: Dow Jones Up But Nvidia Falls On New China Threat; Cathie Wood Loads Up On This Stock Amid 76% Dive (Live Coverage)The Dow Jones Industrial Average reversed higher Monday while other indexes were relatively flat. The Dow moved higher even as Nvidia skidded on a report that China's Huawei is developing a rival... Read More\nView Prior Stock Market Today Updates\n\n\nCurrent Recommended Market Exposure Level [...] MarketSurge\nIBD Live\nLeaderboard\nSwingTrader\

MarketDataTool

In [159]:
import yfinance as yf
from datetime import datetime, timedelta


def MarketData(entity):

    """
    Fetch stock prices and financial data for the past 30 days for a specific company or entity.

    Args   : 
        entity : str
    Return :
        Market data content : str
    """
    
    ticker_response = llm.invoke(f"""
        What are the stock ticker symbol for the following company: {entity}?
        If any company doesn't exist, output None for it.
        Return the ticker only with no extra text.
    """)

    ticker = ticker_response.content.strip()
    print(ticker)


    stock = yf.Ticker(ticker)
    data = stock.history(period="30d")
    summary = f"Stock data for {ticker} i.e {entity} over the last 30 days:\n"

    for date,row in data.iterrows():
        date_str = date.strftime("%b %d, %Y")
        open_price = row["Open"]
        close_price = row["Close"]
        volume = row["Volume"]    
        summary += f"Date: {date_str}, Open: ${open_price:.2f}, Close: ${close_price:.2f}, Volume: {volume}\n"

    return summary

In [160]:
MarketData("Apple")

AAPL


'Stock data for AAPL i.e Apple over the last 30 days:\nDate: Mar 17, 2025, Open: $213.31, Close: $214.00, Volume: 48073400.0\nDate: Mar 18, 2025, Open: $214.16, Close: $212.69, Volume: 42432400.0\nDate: Mar 19, 2025, Open: $214.22, Close: $215.24, Volume: 54385400.0\nDate: Mar 20, 2025, Open: $213.99, Close: $214.10, Volume: 48862900.0\nDate: Mar 21, 2025, Open: $211.56, Close: $218.27, Volume: 94127800.0\nDate: Mar 24, 2025, Open: $221.00, Close: $220.73, Volume: 44299500.0\nDate: Mar 25, 2025, Open: $220.77, Close: $223.75, Volume: 34493600.0\nDate: Mar 26, 2025, Open: $223.51, Close: $221.53, Volume: 34466100.0\nDate: Mar 27, 2025, Open: $221.39, Close: $223.85, Volume: 37094800.0\nDate: Mar 28, 2025, Open: $221.67, Close: $217.90, Volume: 39818600.0\nDate: Mar 31, 2025, Open: $217.01, Close: $222.13, Volume: 65299300.0\nDate: Apr 01, 2025, Open: $219.81, Close: $223.19, Volume: 36412700.0\nDate: Apr 02, 2025, Open: $221.32, Close: $223.89, Volume: 35905900.0\nDate: Apr 03, 2025, Op

NewsTool

In [161]:
import requests

os.environ["GNEWS_API_KEY"] = os.getenv("GNEWS_API_KEY")

def NewsTool(entity):
    
    """
    Retrieve the latest news headlines related to a company or entity.

    Args   : 
        entity : str
     :
        news related content : str
    """
        
    url = f'https://gnews.io/api/v4/search?q={entity}&lang=en&token={os.environ["GNEWS_API_KEY"]}'
    response = requests.get(url)
    data = response.json()
    if data.get("articles"):
        headlines = [article["title"] for article in data["articles"]]
        content = f"News for {entity}:\n" + "\n".join(headlines) + "\n"
    else:
        content = f"No news found for {entity}.\n"
    return content


In [162]:
NewsTool("Apple")

"News for Apple:\nAirPods Pro 3’s best upgrade could be the gift that keeps giving\nApple Watch SE 3: 3 reasons to consider upgrading, and 3 reasons you might want to skip\nMassive Apple sale is live at Amazon, including record-low price on MacBooks, iPads, and AirPods\nApple Seeds Fourth Beta of iOS 18.5 to Developers [Update: Public Beta Available]\nNew Meta XR glasses again tipped to land later this year - well ahead of Apple's rumored AR glasses with Apple Intelligence\nApple’s most affordable iPad is now the cheapest it’s ever been\n3 new Apple TV+ movies and shows you should stream in May 2025\nHow to get the most out of Siri on your Apple Watch\nCriminals are pretending to be Microsoft, Google, and Apple in phishing attacks\nApple Watch Series 11 isn’t tempting me to upgrade based on these 5 rumors\n"

RelativeComparingTool

In [163]:
def RelativeComparisionTool(entity : str) -> str:

    """ 
    
    Provide a detailed analysis of a company's market position, including comparisons with competitors.

    Args :
        entity : str
    Return : 
        content : str

    """

    content = ""

    return content
    

In [164]:
tools_available = [
    ToolSchema(
        name="MarketDataTool",
        description="Fetch stock prices and financial data for the past 30 days for a specific company or entity.",
        input_schema="string i.e the entity name"
    ),
    ToolSchema(
        name="NewsTool",
        description="Retrieve the latest news headlines related to a company or entity.",
        input_schema="string i.e the entity name"
    ),
    ToolSchema(
        name="GeneralSearchTool",
        description="Execute a web search using the provided general search query — a well-formed, natural language question intended to retrieve general market or contextual information.",
        input_schema="A general search query string (e.g., 'how is the stock market performing today')"
    ),
    ToolSchema(
        name="SocialMediaTool",
        description= "Collect social media posts about a company or entity from platforms like Reddit, Twitter, and StockTwits.",
        input_schema="string i.e the entity name"
    )
]

# {
#     "name": "RelativeComparisionTool",
#     "description": "Provide a detailed analysis of a company's market position, including comparisons with competitors."
# },

In [165]:
tool_list_text = "\n\n".join([
    f"{tool.name}: {tool.description}\nInputs: {tool.input_schema}"
    for tool in tools_available 
])
tool_list_text

"MarketDataTool: Fetch stock prices and financial data for the past 30 days for a specific company or entity.\nInputs: string i.e the entity name\n\nNewsTool: Retrieve the latest news headlines related to a company or entity.\nInputs: string i.e the entity name\n\nGeneralSearchTool: Execute a web search using the provided general search query — a well-formed, natural language question intended to retrieve general market or contextual information.\nInputs: A general search query string (e.g., 'how is the stock market performing today')\n\nSocialMediaTool: Collect social media posts about a company or entity from platforms like Reddit, Twitter, and StockTwits.\nInputs: string i.e the entity name"

In [166]:
print(refined_query["refined_query"])
print(refined_query["refined_query"].general_search_query)

entities=['Apple', 'Xiomi'] intent='query' goal='stock information' detail_level='Basic' general_search_query=None
None


In [171]:
# User asked: {state["user_query"]}

class ToolCall(BaseModel):
    tool_call : list[ToolToCall]  

def orchetrator(state : MessageState):

    prompt = f"""

        You are a smart assistant specialize in stock market analysis.
        The user's original query has been refined as follows:

        Here are the details of the refined query:
            - Entities: {', '.join(state['refined_query'].entities)}
            - Intent: {state['refined_query'].intent}
            - Goal: {state['refined_query'].goal}
            - Detail level: {state['refined_query'].detail_level}
            - General Search Query: {state['refined_query'].general_search_query}

        Here are the details of tool : {tool_list_text}

      Based on this information, please decide:
      1. What tools should be called?
      2. For each tool, provide reasons for the call.
      3. If there are multiple entities, handle each one appropriately.
      4. Also provide the input fo reach tool call.

      Return a structured response with tool to call , reason and input.
   
    """

    structured_llm = llm.with_structured_output(ToolCall)
    response = structured_llm.invoke(prompt)

    print(response)


In [172]:
print(refined_query)
orchetrator(refined_query)

{'refined_query': Refined(entities=['Apple', 'Xiomi'], intent='query', goal='stock information', detail_level='Basic', general_search_query=None)}


BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool-use>\n{\n\t"tool_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "MarketDataTool"\n\t\t},\n\t\t"parameters": {\n\t\t\t"input": "Apple"\n\t\t}\n\t}\n}\n</tool-use> \n<tool-use>\n{\n\t"tool_call": {\n\t\t"id": "pending",\n\t\t"type": "function",\n\t\t"function": {\n\t\t\t"name": "MarketDataTool"\n\t\t},\n\t\t"parameters": {\n\t\t\t"input": "Xiomi"\n\t\t}\n\t}\n}\n</tool-use> \n'}}